In [12]:
import numpy as np
import os
import cv2
import face_recognition
import datetime
import pickle
import pandas as pd
import time

In [13]:
directory_path = r"C:\Users\Q209\Current projects\Face Recognition_FR\face dataset\face attendance"

In [14]:
os.listdir(directory_path)

['Akshay Kumar_1',
 'Elon Musk_4',
 'Salman Khan_2',
 'Shahrukh Khan_3',
 'Shreyansh_Q209']

In [15]:
for name in os.listdir(directory_path):
#     print(name)
    for img_name in os.listdir(os.path.join(directory_path, name)):
        img_path = os.path.join(directory_path, name, img_name)
        #print(img_path)
    print(name)

Akshay Kumar_1
Elon Musk_4
Salman Khan_2
Shahrukh Khan_3
Shreyansh_Q209


### training face dataset

In [16]:
# Create empty lists to store known face encodings and names
known_face_encodings = []
known_face_names = []

# Loop through each subdirectory in the path, which should be named after the person
for name_id in os.listdir(directory_path):
    for img_name in os.listdir(os.path.join(directory_path, name_id)):
        img_path = os.path.join(directory_path, name_id, img_name)
        img = face_recognition.load_image_file(img_path)
        face_encoding = face_recognition.face_encodings(img)[0]
        known_face_encodings.append(face_encoding)
        known_face_names.append(name_id)
# print(known_face_names)

In [17]:
print(known_face_encodings)
print('*_'*50)
print(known_face_names)

[array([-0.0698335 ,  0.09373073, -0.0516231 , -0.0309465 , -0.06531221,
        0.04900357, -0.07453755, -0.10832311,  0.16024615,  0.00574016,
        0.13213718, -0.10081818, -0.27199671,  0.05556783, -0.05313336,
        0.05359807, -0.17373516, -0.11276411, -0.12735072, -0.16348894,
        0.04950683,  0.02980918,  0.00863054,  0.06025308, -0.25824523,
       -0.23820394, -0.08286903, -0.23029189,  0.03711396, -0.15591976,
       -0.03366732, -0.0508469 , -0.20177636, -0.04728962,  0.03120061,
        0.04352786, -0.01214946,  0.02029229,  0.17059261,  0.00375216,
       -0.09465219,  0.12051569,  0.08627129,  0.26930749,  0.16602631,
        0.16713047, -0.02022916, -0.04467944,  0.12813117, -0.28048322,
        0.10434863,  0.14663085,  0.08502561,  0.03862558,  0.1602636 ,
       -0.22583078, -0.03869641,  0.108124  , -0.14809017,  0.20429347,
        0.08581584,  0.03513102, -0.06807346, -0.04003466,  0.23154776,
        0.10941693, -0.13546325, -0.09450044,  0.09638701, -0.1

### Save the face encodings into pickle file 

In [18]:
# Save the face encodings to a pickle file for future use
with open('known_face_encodings.pkl', 'wb') as f:
    pickle.dump(known_face_encodings, f)

### Load the saved pickle file

In [19]:
# Load the face encodings from the pickle file
with open('known_face_encodings.pkl', 'rb') as f:
    known_face_encodings = pickle.load(f)

In [20]:
# img_path = 'path/to/input/image'
# img = face_recognition.load_image_file(img_path)
# face_locations = face_recognition.face_locations(img)
# face_encodings = face_recognition.face_encodings(img, face_locations)

# for face_encoding in face_encodings:
#     matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
#     name_id = "Unknown"
#     if True in matches:
#         first_match_index = matches.index(True)
#         name = known_face_names[first_match_index]
#         attendance[name] = 1
        
# # Save attendance in a CSV file with the current date and time as the file name
# now = datetime.now()
# date_string = now.strftime('%Y-%m-%d_%H-%M-%S')
# filename = f'attendance_{date_string}.csv'


In [21]:
#Initialize the attendance dataframe
col_names = ['Name_ID', 'Date', 'Time']
attendance = pd.DataFrame(columns=col_names)

# Initialize the font for labeling the recognized faces
font = cv2.FONT_HERSHEY_DUPLEX

# Define the colors for the rectangles
green = (0, 255, 0)
red = (0, 0, 255)

In [22]:
# Initialize the webcam or load a video file
cap = cv2.VideoCapture(0)  # Replace 0 with the path to a video file if you're using a file

while True:
    # Capture a frame from the video
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert the image from BGR (OpenCV's default) to RGB
    rgb_frame = frame[:, :, ::-1]
    
    # Find all faces in the image
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
    
    
    # Loop through each face in the image
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        # Compare the face encoding to the known face encodings
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        
        
        # Set the color and name for the rectangle based on whether the face is known or unknown
        if True in matches:
            first_match_index = matches.index(True)
            name_id = known_face_names[first_match_index]
            color = green
            
            # Get the current date and time
            now = datetime.datetime.now()
            date = now.strftime('%d-%m-%Y')
            time = now.strftime('%H:%M:%S')
            
            # Add a row to the attendance dataframe with the person's name and ID, and the current date and time
            attendance = attendance.append({'Name_ID': name_id, 'Date': date, 'Time': time}, ignore_index=True)
            # Remove duplicates based on Id and Date columns
            attendance = attendance.drop_duplicates(subset=['Name_ID','Date'],keep='first')
            # Save attendance record to CSV file
            attendance.to_csv('Attendance_update.csv',index=False,header=True)
            
        else:
            name_id = 'Unknown'
            color = red
            
        # Draw the rectangle around the face and label it with the name
        cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
        cv2.rectangle(frame, (left, bottom - 25), (right, bottom), color, cv2.FILLED)
        cv2.putText(frame, name_id, (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1) 
        
        
    # Display the resulting image
    cv2.imshow('Video', frame)
    # Wait for a key press to exit the program
    k = cv2.waitKey(100) & 0xff 
    if k == 27: # Press 'ESC' for exiting video
        break
    elif (k == ord('c')):
        # Check if the directory exists and create it if it doesn't
        if not os.path.exists("CapturedImages"):
            os.mkdir("CapturedImages")
        # Save the captured image into the datasets folder
        cv2.imwrite(f'CapturedImages/{name_id}_{time.strftime("%d%m%Y-%H%M")}.jpg', frame)
        #cv2.imwrite(r'C:\Users\Q209\Current projects\Face Recognition_jupyter/User_" + individual_name + ".jpg"',frame)
        break

        
# Do a bit of cleanup
print("\n [INFO] Exiting Program ")    
# Release handle to the webcam
cap.release()
cv2.destroyAllWindows()

C:\Users\Q209\AppData\Local\Temp\ipykernel_15268\642284359.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  attendance = attendance.append({'Name_ID': name_id, 'Date': date, 'Time': time}, ignore_index=True)
C:\Users\Q209\AppData\Local\Temp\ipykernel_15268\642284359.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  attendance = attendance.append({'Name_ID': name_id, 'Date': date, 'Time': time}, ignore_index=True)
C:\Users\Q209\AppData\Local\Temp\ipykernel_15268\642284359.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  attendance = attendance.append({'Name_ID': name_id, 'Date': date, 'Time': time}, ignore_index=True)
C:\Users\Q209\AppData\Local\Temp\ipykernel_15268\642284359.py:36: FutureWarning: The frame.append

C:\Users\Q209\AppData\Local\Temp\ipykernel_15268\642284359.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  attendance = attendance.append({'Name_ID': name_id, 'Date': date, 'Time': time}, ignore_index=True)
C:\Users\Q209\AppData\Local\Temp\ipykernel_15268\642284359.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  attendance = attendance.append({'Name_ID': name_id, 'Date': date, 'Time': time}, ignore_index=True)
C:\Users\Q209\AppData\Local\Temp\ipykernel_15268\642284359.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  attendance = attendance.append({'Name_ID': name_id, 'Date': date, 'Time': time}, ignore_index=True)
C:\Users\Q209\AppData\Local\Temp\ipykernel_15268\642284359.py:36: FutureWarning: The frame.append

C:\Users\Q209\AppData\Local\Temp\ipykernel_15268\642284359.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  attendance = attendance.append({'Name_ID': name_id, 'Date': date, 'Time': time}, ignore_index=True)
C:\Users\Q209\AppData\Local\Temp\ipykernel_15268\642284359.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  attendance = attendance.append({'Name_ID': name_id, 'Date': date, 'Time': time}, ignore_index=True)
C:\Users\Q209\AppData\Local\Temp\ipykernel_15268\642284359.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  attendance = attendance.append({'Name_ID': name_id, 'Date': date, 'Time': time}, ignore_index=True)
C:\Users\Q209\AppData\Local\Temp\ipykernel_15268\642284359.py:36: FutureWarning: The frame.append


 [INFO] Exiting Program 
